# Chapter 15: Building AI Agents - Interactive Examples

This notebook demonstrates the practical implementation of AI agents discussed in Chapter 15.
We'll build working examples from simple loops to complex multi-step workflows.

## Cell 1: Setup and Imports

Install required packages and import libraries.

In [ ]:
# Install required packages
!pip install anthropic python-dotenv pydantic networkx dataclasses-json -q

import os
import json
import time
import asyncio
from datetime import datetime
from dataclasses import dataclass, asdict
from typing import Dict, List, Optional, Callable, Any
from enum import Enum
import networkx as nx
from collections import Counter
import anthropic

# Set up API key
api_key = os.environ.get("ANTHROPIC_API_KEY", "sk-your-key-here")
client = anthropic.Anthropic(api_key=api_key)

## Cell 2: Simple Loop Agent

The most basic agent pattern: perceive, act, repeat.

In [ ]:
class SimpleLoopAgent:
    """Basic agent that uses a simple loop pattern."""
    
    def __init__(self, model="claude-3-5-sonnet-20241022"):
        self.model = model
        self.conversation = []
        self.tools = self._define_tools()
    
    def _define_tools(self):
        return [
            {
                "name": "calculate",
                "description": "Perform basic arithmetic calculations",
                "input_schema": {
                    "type": "object",
                    "properties": {
                        "operation": {
                            "type": "string",
                            "description": "Operation: add, subtract, multiply, divide"
                        },
                        "a": {"type": "number", "description": "First operand"},
                        "b": {"type": "number", "description": "Second operand"}
                    },
                    "required": ["operation", "a", "b"]
                }
            },
            {
                "name": "get_weather",
                "description": "Get weather for a location",
                "input_schema": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", "description": "City name"}
                    },
                    "required": ["location"]
                }
            }
        ]
    
    def _execute_tool(self, tool_name, tool_input):
        """Execute a tool and return result."""
        if tool_name == "calculate":
            op = tool_input.get("operation")
            a, b = tool_input.get("a"), tool_input.get("b")
            
            if op == "add":
                return a + b
            elif op == "subtract":
                return a - b
            elif op == "multiply":
                return a * b
            elif op == "divide":
                return a / b if b != 0 else "Error: Division by zero"
        
        elif tool_name == "get_weather":
            location = tool_input.get("location")
            # Simulate weather data
            weather_data = {
                "San Francisco": "72°F, Cloudy",
                "New York": "68°F, Rainy",
                "London": "59°F, Overcast",
                "Tokyo": "75°F, Sunny"
            }
            return weather_data.get(location, f"Weather for {location}: 70°F, Partly Cloudy")
        
        return f"Unknown tool: {tool_name}"
    
    def run(self, user_input, max_iterations=5):
        """Run the agent loop."""
        print(f"User: {user_input}")
        
        self.conversation.append({
            "role": "user",
            "content": user_input
        })
        
        for iteration in range(max_iterations):
            # Get agent response
            response = client.messages.create(
                model=self.model,
                max_tokens=1024,
                tools=self.tools,
                messages=self.conversation
            )
            
            # Add assistant response to conversation
            self.conversation.append({
                "role": "assistant",
                "content": response.content
            })
            
            # Check if we're done
            if response.stop_reason == "end_turn":
                # Extract final text response
                for block in response.content:
                    if hasattr(block, 'text'):
                        print(f"\nAssistant: {block.text}")
                        return block.text
            
            # Process tool calls
            if response.stop_reason == "tool_use":
                tool_results = []
                
                for block in response.content:
                    if block.type == "tool_use":
                        tool_name = block.name
                        tool_input = block.input
                        
                        print(f"\nTool: {tool_name}({json.dumps(tool_input)})")
                        result = self._execute_tool(tool_name, tool_input)
                        print(f"Result: {result}")
                        
                        tool_results.append({
                            "type": "tool_result",
                            "tool_use_id": block.id,
                            "content": str(result)
                        })
                
                # Add tool results to conversation
                self.conversation.append({
                    "role": "user",
                    "content": tool_results
                })
        
        return "Max iterations reached"

# Test the simple loop agent
agent = SimpleLoopAgent()
response = agent.run("What's the weather like in San Francisco and New York? Then calculate 25 * 4.")

## Cell 3: Error Classification and Handling

Implementing robust error handling with classification.

In [ ]:
class ErrorType(Enum):
    """Error classification types."""
    TRANSIENT = "transient"      # Temporary, retry likely to succeed
    PERMANENT = "permanent"      # Won't succeed, don't retry
    RESOURCE = "resource"        # Resource exhausted, wait and retry
    USER = "user"                # User input invalid
    EXTERNAL = "external"        # External service issue
    INTERNAL = "internal"        # Internal system error

def classify_error(exception):
    """Classify error type based on exception."""
    if isinstance(exception, TimeoutError):
        return ErrorType.TRANSIENT
    elif isinstance(exception, ValueError):
        return ErrorType.USER
    elif isinstance(exception, MemoryError):
        return ErrorType.RESOURCE
    elif isinstance(exception, KeyError):
        return ErrorType.PERMANENT
    else:
        return ErrorType.INTERNAL

@dataclass
class ErrorContext:
    """Context for logging errors."""
    error_type: ErrorType
    message: str
    original_exception: Exception
    step_id: str
    attempt: int
    timestamp: datetime
    agent_state: dict

class ErrorLogger:
    """Log and track errors during agent execution."""
    
    def __init__(self):
        self.error_history = []
    
    def log_error(self, context: ErrorContext):
        """Log an error with context."""
        self.error_history.append(context)
        print(f"[{context.error_type.value.upper()}] {context.message}")
    
    def get_error_summary(self, step_id=None):
        """Get summary of errors."""
        errors = self.error_history
        if step_id:
            errors = [e for e in errors if e.step_id == step_id]
        
        error_types = Counter(e.error_type.value for e in errors)
        error_steps = Counter(e.step_id for e in errors)
        
        return {
            "total_errors": len(errors),
            "by_type": dict(error_types),
            "by_step": dict(error_steps)
        }

# Demo error handling
logger = ErrorLogger()

# Simulate error scenarios
errors = [
    (ValueError("Invalid input"), "step_1"),
    (TimeoutError("Connection timeout"), "step_2"),
    (MemoryError("Out of memory"), "step_3"),
    (TimeoutError("Connection timeout"), "step_2"),
]

for exc, step_id in errors:
    error_type = classify_error(exc)
    context = ErrorContext(
        error_type=error_type,
        message=str(exc),
        original_exception=exc,
        step_id=step_id,
        attempt=1,
        timestamp=datetime.now(),
        agent_state={}
    )
    logger.log_error(context)

print("\nError Summary:")
print(json.dumps(logger.get_error_summary(), indent=2))

## Cell 4: Retry Mechanism with Exponential Backoff

Implementing resilient retries.

In [ ]:
class RetryPolicy:
    """Policy for retrying failed operations."""
    
    def __init__(self, max_attempts=3, backoff_type="exponential", 
                 initial_delay=1, max_delay=60):
        self.max_attempts = max_attempts
        self.backoff_type = backoff_type
        self.initial_delay = initial_delay
        self.max_delay = max_delay
    
    def get_delay(self, attempt):
        """Calculate delay for given attempt number."""
        if self.backoff_type == "exponential":
            delay = self.initial_delay * (2 ** attempt)
        elif self.backoff_type == "linear":
            delay = self.initial_delay * (attempt + 1)
        else:
            delay = self.initial_delay
        
        return min(delay, self.max_delay)

class RetryExecutor:
    """Execute functions with automatic retry."""
    
    def __init__(self, policy=None):
        self.policy = policy or RetryPolicy()
    
    def execute(self, fn, args=None, kwargs=None):
        """Execute function with retries."""
        args = args or ()
        kwargs = kwargs or {}
        
        last_error = None
        for attempt in range(self.policy.max_attempts):
            try:
                print(f"Attempt {attempt + 1}/{self.policy.max_attempts}...")
                return fn(*args, **kwargs)
            except Exception as e:
                last_error = e
                print(f"  Failed: {str(e)}")
                
                error_type = classify_error(e)
                
                # Don't retry permanent errors
                if error_type == ErrorType.PERMANENT:
                    raise
                
                if attempt < self.policy.max_attempts - 1:
                    delay = self.policy.get_delay(attempt)
                    print(f"  Waiting {delay:.1f}s before retry...")
                    time.sleep(delay)
        
        raise Exception(f"Failed after {self.policy.max_attempts} attempts") from last_error

# Demo: Simulated flaky function
attempt_count = 0

def flaky_operation():
    global attempt_count
    attempt_count += 1
    if attempt_count < 3:
        raise TimeoutError("Connection timeout")
    return "Success!"

# Execute with retry
executor = RetryExecutor(RetryPolicy(max_attempts=4, initial_delay=0.5))
try:
    result = executor.execute(flaky_operation)
    print(f"\nFinal result: {result}")
except Exception as e:
    print(f"\nFailed: {str(e)}")

## Cell 5: State Management

Implementing agent state tracking.

In [ ]:
@dataclass
class AgentState:
    """Explicit state object for agent execution."""
    task_id: str
    user_input: str
    status: str  # "planning", "executing", "complete", "error"
    current_step: int
    plan: List[str]
    results: Dict[int, str]
    errors: List[str]
    created_at: datetime
    updated_at: datetime
    
    def to_dict(self):
        """Convert to dictionary."""
        return {
            "task_id": self.task_id,
            "user_input": self.user_input,
            "status": self.status,
            "current_step": self.current_step,
            "plan": self.plan,
            "results": self.results,
            "errors": self.errors,
            "created_at": self.created_at.isoformat(),
            "updated_at": self.updated_at.isoformat()
        }
    
    def to_json(self):
        """Convert to JSON string."""
        return json.dumps(self.to_dict(), indent=2)

# Demo: Create and manage state
import uuid

state = AgentState(
    task_id=str(uuid.uuid4()),
    user_input="Write a poem about Python",
    status="planning",
    current_step=0,
    plan=[
        "Understand requirements",
        "Generate poem structure",
        "Write poem",
        "Review and refine"
    ],
    results={},
    errors=[],
    created_at=datetime.now(),
    updated_at=datetime.now()
)

print("Initial State:")
print(state.to_json())

# Update state as steps execute
state.status = "executing"
state.current_step = 1
state.results[0] = "Requirements understood"
state.updated_at = datetime.now()

print("\nUpdated State:")
print(state.to_json())

## Cell 6: DAG-Based Workflow Execution

Implementing workflows with dependencies using directed acyclic graphs.

In [ ]:
class DAGWorkflow:
    """Execute workflows defined as directed acyclic graphs."""
    
    def __init__(self):
        self.graph = nx.DiGraph()
        self.results = {}
        self.execution_log = []
    
    def add_step(self, step_id, action, dependencies=None):
        """Add a step to the workflow."""
        self.graph.add_node(step_id, action=action)
        if dependencies:
            for dep in dependencies:
                self.graph.add_edge(dep, step_id)
    
    def validate(self):
        """Validate that the workflow is a valid DAG."""
        if not nx.is_directed_acyclic_graph(self.graph):
            raise ValueError("Workflow contains cycles")
        return True
    
    def execute(self):
        """Execute the workflow in topological order."""
        self.validate()
        
        # Topological sort to get execution order
        execution_order = list(nx.topological_sort(self.graph))
        print(f"Execution order: {' -> '.join(execution_order)}")
        print()
        
        for step_id in execution_order:
            node = self.graph.nodes[step_id]
            action = node["action"]
            
            # Get results from dependencies
            dependencies = list(self.graph.predecessors(step_id))
            inputs = {dep: self.results[dep] for dep in dependencies}
            
            # Execute
            print(f"Executing {step_id}...")
            try:
                result = action(**inputs) if inputs else action()
                self.results[step_id] = result
                self.execution_log.append((step_id, "success", result))
                print(f"  ✓ Result: {result}")
            except Exception as e:
                self.execution_log.append((step_id, "error", str(e)))
                print(f"  ✗ Error: {str(e)}")
                raise
        
        return self.results

# Demo: Create a data pipeline workflow
workflow = DAGWorkflow()

# Define step actions
def extract_data():
    return [1, 2, 3, 4, 5]

def transform_data(data=None):
    if data is None:
        data = []
    return [x * 2 for x in data]

def filter_data(transformed=None):
    if transformed is None:
        transformed = []
    return [x for x in transformed if x > 4]

def analyze_data(filtered=None):
    if filtered is None:
        filtered = []
    return {"sum": sum(filtered), "count": len(filtered), "avg": sum(filtered) / len(filtered) if filtered else 0}

# Build workflow
workflow.add_step("extract", extract_data)
workflow.add_step("transform", transform_data, dependencies=["extract"])
workflow.add_step("filter", filter_data, dependencies=["transform"])
workflow.add_step("analyze", analyze_data, dependencies=["filter"])

# Execute
results = workflow.execute()
print(f"\nFinal Results: {json.dumps(results, indent=2)}")

## Cell 7: Tool Executor with Validation

Building a robust tool execution framework.

In [ ]:
class ToolExecutor:
    """Execute tools with validation and error handling."""
    
    def __init__(self):
        self.tools = {}
        self.execution_history = []
    
    def register_tool(self, definition, handler):
        """Register a tool with definition and handler."""
        self.tools[definition["name"]] = {
            "definition": definition,
            "handler": handler
        }
    
    def execute(self, tool_name, parameters):
        """Execute a tool with parameter validation."""
        if tool_name not in self.tools:
            return {"error": f"Unknown tool: {tool_name}"}
        
        tool_info = self.tools[tool_name]
        
        # Validate parameters
        validation_error = self._validate_parameters(
            parameters, 
            tool_info["definition"]
        )
        if validation_error:
            return {"error": f"Invalid parameters: {validation_error}"}
        
        # Execute with error handling
        try:
            result = tool_info["handler"](**parameters)
            execution_record = {
                "tool": tool_name,
                "parameters": parameters,
                "result": result,
                "status": "success",
                "timestamp": datetime.now().isoformat()
            }
        except Exception as e:
            result = {"error": str(e)}
            execution_record = {
                "tool": tool_name,
                "parameters": parameters,
                "result": result,
                "status": "error",
                "timestamp": datetime.now().isoformat(),
                "exception_type": type(e).__name__
            }
        
        self.execution_history.append(execution_record)
        return result
    
    def _validate_parameters(self, params, tool_definition):
        """Validate parameters against tool definition."""
        required_params = tool_definition.get("input_schema", {}).get("required", [])
        
        for req_param in required_params:
            if req_param not in params:
                return f"Missing required parameter: {req_param}"
        
        return None
    
    def get_history(self):
        """Get execution history."""
        return self.execution_history

# Demo: Register and execute tools
executor = ToolExecutor()

# Register tools
executor.register_tool(
    {
        "name": "multiply",
        "description": "Multiply two numbers",
        "input_schema": {
            "type": "object",
            "properties": {
                "a": {"type": "number"},
                "b": {"type": "number"}
            },
            "required": ["a", "b"]
        }
    },
    lambda a, b: a * b
)

executor.register_tool(
    {
        "name": "concatenate",
        "description": "Concatenate strings",
        "input_schema": {
            "type": "object",
            "properties": {
                "text1": {"type": "string"},
                "text2": {"type": "string"}
            },
            "required": ["text1", "text2"]
        }
    },
    lambda text1, text2: f"{text1} {text2}"
)

# Execute tools
print("Executing multiply(5, 3):")
result1 = executor.execute("multiply", {"a": 5, "b": 3})
print(f"Result: {result1}\n")

print("Executing concatenate('Hello', 'World'):")
result2 = executor.execute("concatenate", {"text1": "Hello", "text2": "World"})
print(f"Result: {result2}\n")

print("Executing multiply with missing parameter:")
result3 = executor.execute("multiply", {"a": 5})
print(f"Result: {result3}\n")

print("Execution History:")
for record in executor.get_history():
    print(f"  {record['tool']}: {record['status']}")

## Cell 8: Decision Making Framework

Implementing multi-criteria decision analysis.

In [ ]:
class MultiCriteriaDecisionAnalysis:
    """MCDA framework for evaluating options against multiple criteria."""
    
    def __init__(self, criteria, weights):
        """
        Initialize MCDA.
        
        Args:
            criteria: List of evaluation functions
            weights: List of weights for each criterion (must sum to 1)
        """
        if len(criteria) != len(weights):
            raise ValueError("Number of criteria must match number of weights")
        
        if abs(sum(weights) - 1.0) > 0.01:
            raise ValueError("Weights must sum to 1")
        
        self.criteria = criteria
        self.weights = weights
    
    def evaluate_option(self, option):
        """Evaluate a single option against all criteria."""
        scores = []
        for criterion in self.criteria:
            score = criterion(option)
            scores.append(score)
        return scores
    
    def decide(self, options):
        """Choose the best option."""
        scores = {}
        detailed_scores = {}
        
        for option in options:
            criterion_scores = self.evaluate_option(option)
            weighted_score = sum(s * w for s, w in zip(criterion_scores, self.weights))
            scores[option] = weighted_score
            detailed_scores[option] = criterion_scores
        
        best_option = max(options, key=lambda o: scores[o])
        
        return {
            "best_option": best_option,
            "scores": scores,
            "detailed_scores": detailed_scores
        }

# Demo: Choose best LLM model
@dataclass
class LLMModel:
    name: str
    accuracy: float  # 0-1
    latency_ms: float
    cost_per_1k: float
    reliability: float  # 0-1

models = [
    LLMModel("GPT-4", 0.95, 2000, 0.03, 0.98),
    LLMModel("Claude 3.5 Sonnet", 0.92, 800, 0.003, 0.99),
    LLMModel("Gemini Pro", 0.90, 1000, 0.0005, 0.95),
]

# Define criteria and weights
criteria = [
    lambda m: m.accuracy,              # Accuracy (weight: 0.4)
    lambda m: 1 / (m.latency_ms / 1000),  # Speed (weight: 0.3) 
    lambda m: 1 / m.cost_per_1k,       # Cost efficiency (weight: 0.2)
    lambda m: m.reliability             # Reliability (weight: 0.1)
]
weights = [0.4, 0.3, 0.2, 0.1]

mdca = MultiCriteriaDecisionAnalysis(criteria, weights)
result = mdca.decide(models)

print("Model Selection Results:")
print(f"Best Choice: {result['best_option'].name}")
print(f"\nScores:")
for model, score in sorted(result['scores'].items(), key=lambda x: x[1], reverse=True):
    print(f"  {model.name}: {score:.3f}")

print(f"\nDetailed Scores (Accuracy, Speed, Cost, Reliability):")
for model, scores in result['detailed_scores'].items():
    print(f"  {model.name}: {[f'{s:.2f}' for s in scores]}")

## Cell 9: Circuit Breaker Pattern

Preventing cascading failures.

In [ ]:
class CircuitState(Enum):
    """Circuit breaker states."""
    CLOSED = "closed"          # Normal operation
    OPEN = "open"              # Failing, reject requests
    HALF_OPEN = "half_open"    # Testing recovery

class CircuitBreaker:
    """Prevent cascading failures using circuit breaker pattern."""
    
    def __init__(self, failure_threshold=5, recovery_timeout=10):
        self.failure_threshold = failure_threshold
        self.recovery_timeout = recovery_timeout
        self.state = CircuitState.CLOSED
        self.failure_count = 0
        self.last_failure_time = None
        self.call_log = []
    
    def call(self, fn, *args, **kwargs):
        """Execute function with circuit breaker protection."""
        if self.state == CircuitState.OPEN:
            if time.time() - self.last_failure_time > self.recovery_timeout:
                print(f"  [Circuit] Transitioning to HALF_OPEN")
                self.state = CircuitState.HALF_OPEN
            else:
                self.call_log.append(("blocked", "circuit open"))
                raise Exception("Circuit breaker is OPEN")
        
        try:
            result = fn(*args, **kwargs)
            self.on_success()
            self.call_log.append(("success", None))
            return result
        except Exception as e:
            self.on_failure()
            self.call_log.append(("failure", str(e)))
            raise
    
    def on_success(self):
        """Handle successful call."""
        self.failure_count = 0
        if self.state == CircuitState.HALF_OPEN:
            print(f"  [Circuit] Recovered - transitioning to CLOSED")
            self.state = CircuitState.CLOSED
    
    def on_failure(self):
        """Handle failed call."""
        self.failure_count += 1
        self.last_failure_time = time.time()
        
        if self.failure_count >= self.failure_threshold:
            print(f"  [Circuit] Failure threshold reached - opening circuit")
            self.state = CircuitState.OPEN
    
    def get_state(self):
        return self.state.value

# Demo: Simulate service with intermittent failures
failing_call_count = 0

def unreliable_service():
    global failing_call_count
    failing_call_count += 1
    
    # Fails first 5 times, then succeeds
    if failing_call_count <= 5:
        raise Exception("Service unavailable")
    return "Success!"

breaker = CircuitBreaker(failure_threshold=3, recovery_timeout=2)

print("Testing circuit breaker:")
for i in range(12):
    print(f"\nCall {i+1}:")
    try:
        result = breaker.call(unreliable_service)
        print(f"  Result: {result}")
    except Exception as e:
        print(f"  Error: {e}")
    print(f"  Circuit state: {breaker.get_state()}")
    
    if i == 6:
        print("\n  [Waiting for recovery timeout...]")
        time.sleep(2.5)

In [ ]:
class PlanningAgent:
    """Agent that plans before executing."""
    
    def __init__(self, model="claude-3-5-sonnet-20241022"):
        self.model = model
    
    def create_plan(self, task):
        """Create a plan for the task."""
        system_prompt = """You are a planning assistant. Given a task, create a detailed step-by-step plan.
        
Format your response as JSON with this structure:
{
  "task": "description of the task",
  "objective": "the main goal",
  "steps": [
    {"id": 1, "action": "specific action", "dependencies": []},
    {"id": 2, "action": "next action", "dependencies": [1]}
  ]
}

Make the plan clear and structured."""
        
        response = client.messages.create(
            model=self.model,
            max_tokens=1024,
            system=system_prompt,
            messages=[{"role": "user", "content": f"Create a plan for: {task}"}]
        )
        
        # Parse response
        response_text = response.content[0].text
        
        # Try to extract JSON
        try:
            plan_json = response_text[response_text.find('{'):response_text.rfind('}')+1]
            plan = json.loads(plan_json)
        except:
            plan = {"raw_response": response_text}
        
        return plan
    
    def execute_plan(self, plan):
        """Execute the plan (simulation)."""
        results = {}
        
        if "steps" in plan:
            for step in plan["steps"]:
                step_id = step.get("id")
                action = step.get("action")
                print(f"Executing step {step_id}: {action}")
                results[step_id] = f"Completed: {action}"
        
        return results

# Demo: Plan a task
agent = PlanningAgent()
print("Creating plan for task: 'Research and summarize benefits of Python programming language'\n")
plan = agent.create_plan("Research and summarize benefits of Python programming language")

print("Plan:")
print(json.dumps(plan, indent=2))

## Cell 11: Monitoring and Metrics

Track agent performance and behavior.

In [ ]:
class AgentMetrics:
    """Track metrics for agent execution."""
    
    def __init__(self):
        self.task_counts = Counter()
        self.task_durations = []
        self.tool_calls = Counter()
        self.errors = Counter()
        self.concurrent_tasks = 0
    
    def record_task(self, status):
        """Record task completion."""
        self.task_counts[status] += 1
    
    def record_duration(self, duration_seconds):
        """Record task duration."""
        self.task_durations.append(duration_seconds)
    
    def record_tool_call(self, tool_name, status):
        """Record tool call."""
        self.tool_calls[f"{tool_name}:{status}"] += 1
    
    def record_error(self, error_type):
        """Record error."""
        self.errors[error_type] += 1
    
    def get_summary(self):
        """Get metrics summary."""
        avg_duration = sum(self.task_durations) / len(self.task_durations) if self.task_durations else 0
        
        return {
            "task_counts": dict(self.task_counts),
            "total_tasks": sum(self.task_counts.values()),
            "success_rate": self.task_counts["success"] / sum(self.task_counts.values()) if sum(self.task_counts.values()) > 0 else 0,
            "avg_duration_seconds": avg_duration,
            "min_duration_seconds": min(self.task_durations) if self.task_durations else 0,
            "max_duration_seconds": max(self.task_durations) if self.task_durations else 0,
            "tool_calls": dict(self.tool_calls),
            "errors": dict(self.errors),
            "total_errors": sum(self.errors.values())
        }

# Demo: Simulate agent execution and metrics
metrics = AgentMetrics()

# Simulate some task execution
for i in range(10):
    if i < 8:
        metrics.record_task("success")
        metrics.record_duration(2.3 + (i % 3) * 0.5)
        metrics.record_tool_call("search", "success")
        metrics.record_tool_call("extract", "success")
    else:
        metrics.record_task("error")
        metrics.record_duration(1.5)
        metrics.record_error("timeout")

print("Agent Metrics Summary:")
summary = metrics.get_summary()
for key, value in summary.items():
    if isinstance(value, float):
        print(f"{key}: {value:.3f}")
    else:
        print(f"{key}: {value}")

## Cell 12: Async/Parallel Workflow Execution

Execute independent steps in parallel.

In [ ]:
class AsyncWorkflow:
    """Execute workflows with async tasks."""
    
    def __init__(self):
        self.steps = {}
        self.results = {}
    
    def add_step(self, step_id, async_action, dependencies=None):
        """Add an async step."""
        self.steps[step_id] = {
            "action": async_action,
            "dependencies": dependencies or []
        }
    
    async def execute(self):
        """Execute workflow asynchronously."""
        completed = set()
        
        while len(completed) < len(self.steps):
            # Find executable steps
            executable = [
                (step_id, step) for step_id, step in self.steps.items()
                if step_id not in completed 
                and all(dep in completed for dep in step["dependencies"])
            ]
            
            if not executable:
                break
            
            # Execute all executable steps in parallel
            tasks = []
            task_map = {}
            
            for step_id, step in executable:
                inputs = {dep: self.results[dep] for dep in step["dependencies"]}
                task = asyncio.create_task(step["action"](**inputs) if inputs else step["action"]())
                tasks.append(task)
                task_map[id(task)] = step_id
            
            # Wait for all tasks
            for task in tasks:
                result = await task
                step_id = task_map[id(task)]
                self.results[step_id] = result
                completed.add(step_id)
        
        return self.results

# Demo: Async workflow
async def task_a():
    await asyncio.sleep(0.5)
    return "Result A"

async def task_b():
    await asyncio.sleep(0.3)
    return "Result B"

async def task_c(a=None, b=None):
    await asyncio.sleep(0.2)
    return f"Combined: {a} and {b}"

async def run_async_demo():
    workflow = AsyncWorkflow()
    
    # A and B execute in parallel, C depends on both
    workflow.add_step("a", task_a)
    workflow.add_step("b", task_b)
    workflow.add_step("c", task_c, dependencies=["a", "b"])
    
    start = time.time()
    results = await workflow.execute()
    duration = time.time() - start
    
    print(f"Async Workflow Results (completed in {duration:.2f}s):")
    for step_id, result in results.items():
        print(f"  {step_id}: {result}")

# Run async demo
await run_async_demo()

## Cell 13: Human-in-the-Loop Agent

Agent that pauses for human approval.

In [ ]:
class HumanApprovalAgent:
    """Agent that requests human approval for actions."""
    
    def __init__(self):
        self.pending_approvals = {}
    
    def request_approval(self, action_description, action_data):
        """Request human approval for an action."""
        approval_id = str(len(self.pending_approvals) + 1)
        
        self.pending_approvals[approval_id] = {
            "id": approval_id,
            "description": action_description,
            "data": action_data,
            "status": "pending",
            "created_at": datetime.now()
        }
        
        print(f"\n[APPROVAL REQUIRED]")
        print(f"ID: {approval_id}")
        print(f"Action: {action_description}")
        print(f"Data: {json.dumps(action_data, indent=2)}")
        
        return approval_id
    
    def approve(self, approval_id):
        """Approve a pending action."""
        if approval_id in self.pending_approvals:
            self.pending_approvals[approval_id]["status"] = "approved"
            print(f"\n✓ Approval {approval_id} granted")
            return True
        return False
    
    def reject(self, approval_id):
        """Reject a pending action."""
        if approval_id in self.pending_approvals:
            self.pending_approvals[approval_id]["status"] = "rejected"
            print(f"\n✗ Approval {approval_id} rejected")
            return True
        return False
    
    def execute_with_approval(self, action, description, data):
        """Execute an action, requesting approval first."""
        approval_id = self.request_approval(description, data)
        
        # Simulate user approval (in real system, would wait for webhook/API call)
        print("\n(Simulating human approval in 2 seconds...)")
        time.sleep(1)
        
        # For demo, auto-approve odd-numbered requests
        if int(approval_id) % 2 == 1:
            self.approve(approval_id)
            result = action()
            print(f"Action executed: {result}")
            return {"status": "approved", "result": result}
        else:
            self.reject(approval_id)
            return {"status": "rejected", "reason": "User rejected"}

# Demo: Human-in-the-loop
agent = HumanApprovalAgent()

# Request approvals for different actions
result1 = agent.execute_with_approval(
    action=lambda: "Email sent to customer",
    description="Send promotional email",
    data={"recipient": "user@example.com", "template": "promo_v2"}
)

result2 = agent.execute_with_approval(
    action=lambda: "Refund processed",
    description="Process refund",
    data={"order_id": "12345", "amount": 99.99}
)

print(f"\nResults: {json.dumps([result1, result2], indent=2)}")

## Cell 14: Custom Agent with All Features

Building a comprehensive agent combining multiple concepts.

In [ ]:
class ComprehensiveAgent:
    """Complete agent with state, tools, metrics, and error handling."""
    
    def __init__(self, model="claude-3-5-sonnet-20241022"):
        self.model = model
        self.executor = ToolExecutor()
        self.metrics = AgentMetrics()
        self.logger = ErrorLogger()
        self._setup_tools()
    
    def _setup_tools(self):
        """Setup available tools."""
        # Register tools
        self.executor.register_tool(
            {
                "name": "search_documentation",
                "description": "Search documentation for information",
                "input_schema": {
                    "type": "object",
                    "properties": {
                        "query": {"type": "string"}
                    },
                    "required": ["query"]
                }
            },
            lambda query: f"Documentation for '{query}': Python is a versatile language..."
        )
        
        self.executor.register_tool(
            {
                "name": "analyze_code",
                "description": "Analyze code for issues",
                "input_schema": {
                    "type": "object",
                    "properties": {
                        "code": {"type": "string"}
                    },
                    "required": ["code"]
                }
            },
            lambda code: {"issues": [], "quality_score": 0.85}
        )
    
    def run_task(self, task_description):
        """Run a task with complete monitoring."""
        task_id = str(len(self.logger.error_history) + 1)
        start_time = time.time()
        
        try:
            print(f"[Task {task_id}] Starting: {task_description}")
            
            # Simulate some work
            result = f"Completed: {task_description}"
            
            duration = time.time() - start_time
            self.metrics.record_task("success")
            self.metrics.record_duration(duration)
            
            print(f"[Task {task_id}] Success ({duration:.2f}s)")
            return {"status": "success", "result": result}
            
        except Exception as e:
            duration = time.time() - start_time
            self.metrics.record_task("error")
            self.metrics.record_error(type(e).__name__)
            
            context = ErrorContext(
                error_type=classify_error(e),
                message=str(e),
                original_exception=e,
                step_id=task_id,
                attempt=1,
                timestamp=datetime.now(),
                agent_state={}
            )
            self.logger.log_error(context)
            
            print(f"[Task {task_id}] Error ({duration:.2f}s): {str(e)}")
            return {"status": "error", "error": str(e)}
    
    def get_report(self):
        """Get comprehensive agent report."""
        return {
            "metrics": self.metrics.get_summary(),
            "errors": self.logger.get_error_summary(),
            "tool_executions": len(self.executor.get_history())
        }

# Demo: Comprehensive agent
agent = ComprehensiveAgent()

# Run several tasks
tasks = [
    "Analyze customer feedback",
    "Generate report",
    "Search documentation",
    "Fix bug in code"
]

for task in tasks:
    agent.run_task(task)
    time.sleep(0.5)

print("\n" + "="*50)
print("Agent Report:")
print("="*50)
report = agent.get_report()
print(json.dumps(report, indent=2))

## Cell 15: Interactive Agent Playground

An interactive environment to experiment with agent concepts.

In [ ]:
class AgentPlayground:
    """Interactive environment to test agent concepts."""
    
    def __init__(self):
        self.simple_agent = SimpleLoopAgent()
        self.planning_agent = PlanningAgent()
        self.comprehensive_agent = ComprehensiveAgent()
        self.session_log = []
    
    def test_simple_loop(self, query):
        """Test simple loop agent."""
        print(f"\n[Simple Loop Agent] Query: {query}")
        print("-" * 50)
        result = self.simple_agent.run(query, max_iterations=2)
        self.session_log.append(("simple_loop", query, result))
        return result
    
    def test_planning(self, task):
        """Test planning agent."""
        print(f"\n[Planning Agent] Task: {task}")
        print("-" * 50)
        plan = self.planning_agent.create_plan(task)
        self.session_log.append(("planning", task, plan))
        return plan
    
    def test_error_handling(self):
        """Test error handling capabilities."""
        print("\n[Error Handling Test]")
        print("-" * 50)
        
        errors_to_test = [
            ValueError("Invalid input"),
            TimeoutError("Request timeout"),
            MemoryError("Out of memory")
        ]
        
        results = []
        for error in errors_to_test:
            error_type = classify_error(error)
            print(f"  {error.__class__.__name__}: {error_type.value}")
            results.append((error.__class__.__name__, error_type.value))
        
        self.session_log.append(("error_handling", "test", results))
        return results
    
    def get_session_summary(self):
        """Get session summary."""
        return {
            "total_tests": len(self.session_log),
            "tests_run": [log[0] for log in self.session_log]
        }

# Create playground and run demonstrations
playground = AgentPlayground()

print("\n" + "="*50)
print("AGENT PLAYGROUND - Interactive Demonstrations")
print("="*50)

# Test 1: Simple loop agent
print("\nTest 1: Simple Loop Agent")
playground.test_simple_loop("What is 15 + 27?")

# Test 2: Error classification
print("\n\nTest 2: Error Handling")
playground.test_error_handling()

# Test 3: Planning agent
print("\n\nTest 3: Planning Agent")
playground.test_planning("Write a Python function to calculate fibonacci numbers")

# Summary
print("\n" + "="*50)
print("Session Summary:")
print("="*50)
summary = playground.get_session_summary()
print(json.dumps(summary, indent=2))
print(f"\n✓ Playground tests completed successfully!")

## Summary

This notebook has demonstrated:

1. **Simple Loop Agents**: Basic agent pattern with tool use and iterative execution
2. **Error Classification & Handling**: Categorizing errors and implementing recovery strategies
3. **Retry Mechanisms**: Exponential backoff and retry policies
4. **State Management**: Tracking agent execution state
5. **DAG Workflows**: Complex workflows with dependencies
6. **Tool Execution Framework**: Registering and executing tools with validation
7. **Decision Making**: Multi-criteria decision analysis (MCDA)
8. **Circuit Breaker Pattern**: Preventing cascading failures
9. **Planning Agents**: Creating explicit plans before execution
10. **Monitoring & Metrics**: Tracking agent performance
11. **Async Workflows**: Parallel execution of independent steps
12. **Human-in-the-Loop**: Requesting human approval for actions
13. **Comprehensive Agent**: Integrating all components
14. **Interactive Playground**: Testing agent concepts

These examples provide a foundation for building production-ready AI agents. Combine these patterns to solve complex problems in your domain.